# Data Wrangling for medications.csv
This notebook processes the `medications.csv` by performing the following steps:
- Inspecting and cleaning the data
- Handling missing values and inconsistencies
- Transforming the data (e.g., creating new features)
- Saving the processed data to a new file

In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import sys
from pprint import pprint

from IPython.display import display, JSON

In [2]:
# Standardizing Column Names
def func_rename_and_cast_columns(df, column_mappings):
    '''
    Applies column_mappings to the columns of input dataframe

    df: Input dataframe
    column_mappings: dictionary containing mappings for column names        
    '''
    list_unnormalised_colnames = df.columns.to_list()
    for unnormalised_colname in list_unnormalised_colnames:
        normalised_colname = column_mappings[unnormalised_colname]['normalised_colname']
        col_type = column_mappings[unnormalised_colname]['type']
        print(f"renaming column: {unnormalised_colname} to {normalised_colname} and casting type to: {col_type}")
        df.rename(columns={unnormalised_colname: normalised_colname}, inplace=True)
        if col_type == 'str':
            # Fill NAs and empty strings to "unknowns"
            if df[normalised_colname].isnull().any() or df[normalised_colname].isna().any():
                print(f"column: {normalised_colname} contains null values thus filling with unknown")
                df[normalised_colname] = df[normalised_colname].fillna("Unknown")
            df[normalised_colname] = df[normalised_colname].astype(str)
            print(f"{normalised_colname} casted to type(str)")
        if col_type == 'datetimestamp':
            # df[normalised_colname] = pd.to_datetime(df[normalised_colname], errors='coerce')
            df[normalised_colname] = pd.to_datetime(df[normalised_colname], errors='coerce') 
            df[normalised_colname] = df[normalised_colname].dt.tz_localize(None) # Retaining source timeformat
            print(f"{normalised_colname} casted to datetime")
        if col_type == 'float':
            df[normalised_colname] = pd.to_numeric(df[normalised_colname], errors='coerce')
            print(f"{normalised_colname} casted to numeric")
        print('-'*5)
    
    return df 

In [3]:
"""
This cell checks whether the file '../../data/processed_data/processed_patients.csv' exists.
If the file does not exist, it prints a message prompting the user to run 'etl/notebooks/etl_patients.ipynb'.
This is useful for ensuring that the necessary preprocessing step has been completed before running the script.
Exception handling is included to capture any unexpected errors.
"""
try:
    filepath_patients_csv = '../../data/processed_data/processed_patients.csv' # READ For join operations
    if not os.path.exists(filepath_patients_csv):
        print(f"{filepath_patients_csv} does not exist. Please run the etl/notebooks/etl_patients.ipynb first.")
        sys.exit()
    else:
        print(f"{filepath_patients_csv} exists. Proceed with the processing.")
except Exception as e:
    print(f"An error occurred: {e}")

../../data/processed_data/processed_patients.csv exists. Proceed with the processing.


In [4]:
# Read Input csv
filepath_csv = '../../data/raw_data/medications.csv' 
# Write processed CSV
filepath_output = '../../data/processed_data/processed_medications.csv' 
# Read medications.yaml, it is used to clean column names and apply relevant types to columns
filepath_yaml = '../../config/medications.yaml' 

# Load Dataframe
df = pd.read_csv(filepath_csv)
df_patients = pd.read_csv(filepath_patients_csv)

# Load YAML column mappings 
with open(filepath_yaml, "r") as file:
    dict_column_mappings = yaml.safe_load(file)
    
# Display initial dataset information
print('Initial Dataset Info:')
df.info()
df.head(5)

Initial Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5605 entries, 0 to 5604
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   START              5605 non-null   object 
 1   STOP               5360 non-null   object 
 2   PATIENT            5605 non-null   object 
 3   PAYER              5605 non-null   object 
 4   ENCOUNTER          5605 non-null   object 
 5   CODE               5605 non-null   int64  
 6   DESCRIPTION        5605 non-null   object 
 7   BASE_COST          5605 non-null   float64
 8   PAYER_COVERAGE     5605 non-null   float64
 9   DISPENSES          5605 non-null   int64  
 10  TOTALCOST          5605 non-null   float64
 11  REASONCODE         4334 non-null   float64
 12  REASONDESCRIPTION  4334 non-null   object 
dtypes: float64(4), int64(2), object(7)
memory usage: 569.4+ KB


,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2015-09-28T11:02:48Z,2015-10-15T09:04:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,857005,Acetaminophen 325 MG / HYDROcodone Bitartrate ...,2.51,0.0,1,2.51,NaN,NaN
1,2015-09-28T11:02:48Z,2015-10-31T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,310965,Ibuprofen 200 MG Oral Tablet,365.90,0.0,1,365.90,NaN,NaN
2,2005-11-08T20:24:07Z,NaN,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,e03e23c9-4df1-3eb6-a62d-f70f02301496,d1cea2e5-1735-089f-c72f-22ad16976663,310325,ferrous sulfate 325 MG Oral Tablet,0.15,0.0,50,7.50,NaN,NaN
3,2020-10-30T11:02:48Z,2020-11-20T11:02:48Z,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,794baa15-fe5e-c061-e188-ad59022aeea5,313782,Acetaminophen 325 MG Oral Tablet,153.58,0.0,1,153.58,NaN,NaN
4,2008-08-27T00:53:03Z,2008-08-27T00:53:03Z,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,8a84efee-6fd7-f5b3-8816-9a1c60e720be,1535362,sodium fluoride 0.0272 MG/MG Oral Gel,129.94,0.0,1,129.94,66383009.0,Gingivitis (disorder)


# Data Cleaning and Data Quality Checks

1. Standardise column names
2. Apply relevant types
3. Fill null values with relevant values
4. Perform the logical testing (start_time < end_time)

In [5]:
display(JSON(dict_column_mappings['columns']))

<IPython.core.display.JSON object>

In [6]:
# Standardise column names (data cleaning, filling null values, apply relevant types using YAML)
df = func_rename_and_cast_columns(df, dict_column_mappings['columns'])

renaming column: START to start_time and casting type to: datetimestamp
start_time casted to datetime
-----
renaming column: STOP to stop_time and casting type to: datetimestamp
stop_time casted to datetime
-----
renaming column: PATIENT to patient_id and casting type to: str
patient_id casted to type(str)
-----
renaming column: PAYER to payer_id and casting type to: str
payer_id casted to type(str)
-----
renaming column: ENCOUNTER to encounter_id and casting type to: str
encounter_id casted to type(str)
-----
renaming column: CODE to code and casting type to: str
code casted to type(str)
-----
renaming column: DESCRIPTION to medicine_description and casting type to: str
medicine_description casted to type(str)
-----
renaming column: BASE_COST to base_cost and casting type to: float
base_cost casted to numeric
-----
renaming column: PAYER_COVERAGE to payer_coverage and casting type to: float
payer_coverage casted to numeric
-----
renaming column: DISPENSES to medicine_dispenses and cas

In [7]:
# Display the dataframe
df.head(5)


,start_time,stop_time,patient_id,payer_id,encounter_id,code,medicine_description,base_cost,payer_coverage,medicine_dispenses,total_cost_of_medicine,reason_code_for_medication,reason_description_for_medication
0,2015-09-28 11:02:48,2015-10-15 09:04:48,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,857005,Acetaminophen 325 MG / HYDROcodone Bitartrate ...,2.51,0.0,1,2.51,Unknown,Unknown
1,2015-09-28 11:02:48,2015-10-31 11:02:48,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,310965,Ibuprofen 200 MG Oral Tablet,365.90,0.0,1,365.90,Unknown,Unknown
2,2005-11-08 20:24:07,NaT,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,e03e23c9-4df1-3eb6-a62d-f70f02301496,d1cea2e5-1735-089f-c72f-22ad16976663,310325,ferrous sulfate 325 MG Oral Tablet,0.15,0.0,50,7.50,Unknown,Unknown
3,2020-10-30 11:02:48,2020-11-20 11:02:48,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,794baa15-fe5e-c061-e188-ad59022aeea5,313782,Acetaminophen 325 MG Oral Tablet,153.58,0.0,1,153.58,Unknown,Unknown
4,2008-08-27 00:53:03,2008-08-27 00:53:03,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,8a84efee-6fd7-f5b3-8816-9a1c60e720be,1535362,sodium fluoride 0.0272 MG/MG Oral Gel,129.94,0.0,1,129.94,66383009.0,Gingivitis (disorder)


# Data Quality
1. Remove Duplicates
2. See distinct values

In [8]:
# Check for duplicate rows
print(f" (INEFFICIENT FOR LARGE DATAFRAMES) Total length of dataframe BEFORE removing duplicates: {len(df)}")
df = df.drop_duplicates()
print(f" (INEFFICIENT FOR LARGE DATAFRAMES) Total length of dataframe AFTER removing duplicates: {len(df)}")

# Validate categorical columns
categorical_columns = ['medicine_description', 'reason_description_for_medication']
for col in categorical_columns:
    distinct_vals = df[col].unique() 
    tot_len = len(distinct_vals)
    if tot_len < 12:
        print(f'Unique values in {col}: {df[col].unique()}')
    else:
        print(f'Unique values too large for display: {col}: has total {tot_len} distinct values')



 (INEFFICIENT FOR LARGE DATAFRAMES) Total length of dataframe BEFORE removing duplicates: 5605
 (INEFFICIENT FOR LARGE DATAFRAMES) Total length of dataframe AFTER removing duplicates: 5605
Unique values too large for display: medicine_description: has total 121 distinct values
Unique values too large for display: reason_description_for_medication: has total 37 distinct values


# Data Engineering
1. Do the logical test (start_time < stop_time)
2. Impute new columns like length of stay

In [9]:
# Logical Testing
# end_time > start_time
valid_dates = df[df['stop_time'] >= df['start_time']]
invalid_dates = df[df['stop_time'] < df['start_time']]
print(f"total entries in df: {len(df)}, valid_dates: {len(valid_dates)}, invalid_dates:{len(invalid_dates)}  ")

# Length of Stay in hours
df['length_of_medication_in_days'] = (df['stop_time'] - df['start_time']).dt.days / 365.0

# Recovered or medicine ended 
df['medicine_ended_or_recovered'] = df['stop_time'].apply(lambda x: 1 if x else 0)

total entries in df: 5605, valid_dates: 5360, invalid_dates:0  


In [10]:
# len(df[df['medicine_ended_or_recovered'] == 1])
# len(df['encounter_id'].unique())

# Left Join with patients.csv to include:
1. Demographics of patient
2. Other relevant information of patient
3. Impute age of patient
4. Impute age category of patient

In [11]:
df_patients.head(5)

,id,birthdate,deathdate,ssn,drivers,passport,prefix,firstname,middlename,lastname,...,state,county,fips,zip,lat,lon,healthcare_expenses,healthcare_coverage,income,income_category
0,30a6452c-4297-a1ac-977a-6a23237c7b46,1994-02-06,NaN,999-52-8591,S99996852,X47758697X,Mr.,Joshua658,Alvin56,Kunde533,...,Massachusetts,Norfolk County,25021.0,2184,42.211142,-71.045802,56904.96,18019.99,100511,high-income
1,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,1968-08-06,2009-12-11,999-75-3953,S99993577,X28173268X,Mr.,Bennie663,Unknown,Ebert178,...,Massachusetts,Norfolk County,25021.0,2184,42.255420,-70.971016,124024.12,1075.06,49737,medium-income
2,7179458e-d6e3-c723-2530-d4acfe1c2668,2008-12-21,NaN,999-70-1925,Unknown,Unknown,Unknown,Hunter736,Mckinley734,Gerlach374,...,Massachusetts,Plymouth County,Unknown,0,41.648292,-70.850619,45645.06,6154.94,133816,high-income
3,37c177ea-4398-fb7a-29fa-70eb3d673876,1994-01-27,NaN,999-27-9779,S99995100,X83694889X,Mrs.,Carlyn477,Florencia449,Williamson769,...,Massachusetts,Plymouth County,Unknown,0,41.789096,-70.711616,12895.15,659951.61,17382,low-income
4,0fef2411-21f0-a269-82fb-c42b55471405,2019-07-27,NaN,999-50-8977,Unknown,Unknown,Unknown,Robin66,Jeramy610,Gleichner915,...,Massachusetts,Essex County,Unknown,0,42.734183,-70.976410,18500.02,5493.57,52159,medium-income


In [12]:
# Left Join with Patients
patient_columns_needed = ['id', 'birthdate', 'marital', 'race', 'ethnicity', 'gender', 'income', 'income_category']
df_patients = df_patients[patient_columns_needed]
# df_patients.head(5)
print(f"len of df BEFORE left-join: {len(df)}")
df = pd.merge(df, df_patients, how='left', left_on='patient_id', right_on='id')

# remove 'id' from right df
df.drop(['id'], axis=1, inplace=True)

#results.head(5)
print(f"len of df AFTER left-join: {len(df)}")

# Impute Age of Patient
df['birthdate'] = pd.to_datetime(df['birthdate'], errors='coerce')
df['start_time'] = pd.to_datetime(df['start_time'], errors='coerce')
df['age_of_patient'] = (df['start_time'] - df['birthdate']).dt.days / 365.0

# Categorize Age
bins = [0, 17, 65, float("inf")]
labels = ["children", "adult", "senior"]

# Create a new column for income category
df["age_category"] = pd.cut(df["age_of_patient"], bins=bins, labels=labels, right=False)
# df.dtypes
# df.head(5)

len of df BEFORE left-join: 5605
len of df AFTER left-join: 5605


In [13]:
df.head(5)

,start_time,stop_time,patient_id,payer_id,encounter_id,code,medicine_description,base_cost,payer_coverage,medicine_dispenses,...,medicine_ended_or_recovered,birthdate,marital,race,ethnicity,gender,income,income_category,age_of_patient,age_category
0,2015-09-28 11:02:48,2015-10-15 09:04:48,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,857005,Acetaminophen 325 MG / HYDROcodone Bitartrate ...,2.51,0.0,1,...,1,1994-02-06,M,white,nonhispanic,M,100511,high-income,21.654795,adult
1,2015-09-28 11:02:48,2015-10-31 11:02:48,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,953c5138-ce17-4084-3432-1ac23f184528,310965,Ibuprofen 200 MG Oral Tablet,365.90,0.0,1,...,1,1994-02-06,M,white,nonhispanic,M,100511,high-income,21.654795,adult
2,2005-11-08 20:24:07,NaT,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,e03e23c9-4df1-3eb6-a62d-f70f02301496,d1cea2e5-1735-089f-c72f-22ad16976663,310325,ferrous sulfate 325 MG Oral Tablet,0.15,0.0,50,...,1,1968-08-06,D,white,nonhispanic,M,49737,medium-income,37.282192,adult
3,2020-10-30 11:02:48,2020-11-20 11:02:48,30a6452c-4297-a1ac-977a-6a23237c7b46,d31fccc3-1767-390d-966a-22a5156f4219,794baa15-fe5e-c061-e188-ad59022aeea5,313782,Acetaminophen 325 MG Oral Tablet,153.58,0.0,1,...,1,1994-02-06,M,white,nonhispanic,M,100511,high-income,26.747945,adult
4,2008-08-27 00:53:03,2008-08-27 00:53:03,34a4dcc4-35fb-6ad5-ab98-be285c586a4f,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,8a84efee-6fd7-f5b3-8816-9a1c60e720be,1535362,sodium fluoride 0.0272 MG/MG Oral Gel,129.94,0.0,1,...,1,1968-08-06,D,white,nonhispanic,M,49737,medium-income,40.084932,adult


In [14]:
# Save the cleaned and transformed dataset
df.to_csv(filepath_output, index=False)
print(f'Processed data saved to {filepath_output}')

Processed data saved to ../../data/processed_data/processed_medications.csv
